<a href="https://colab.research.google.com/github/AlexanderChiuluvB/AlexDeepLearning/blob/master/TensorflowAlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from datetime import datetime
import tensorflow
import time
import tensorflow as tf
import math

In [0]:
batch_size=32
num_batches = 100
def printActivation(t):
  """
  input: tensor t
  
  """
  
  print(t.op.name,' ',t.get_shape().as_list())

In [0]:
def fcLayer(x,inputD,outputD,reluFlag,name):
  """Fully Connected"""
  with tf.variable_scope(name) as scope:
    w = tf.get_variable("w",shape=[inputD,outputD],dtype="float")
    b = tf.get_variable("b",[outputD],dtype="float")
    out = tf.nn.xw_plus_b(x,w,b,name=scope.name)
    if reluFlag:
      return tf.nn.relu(out)
    else:
      return out

In [0]:
#网络结构

def inference(images):
  
  parameters = []
  #第一层卷积层
  with tf.name_scope('conv1') as scope:
    #初始化内核
    #tf.truncated_normal　截取的正态分布函数，用于初始化卷积核的参数kernel，卷积核尺寸11x11,颜色通道为３，卷积核数量为64
    kernel = tf.Variable(tf.truncated_normal([11,11,3,64],dtype=tf.float32,stddev=1e-1),name='weights')
    #conv2d　strides参数是一个长度为４的列表，[1,x,y,1] x,y,分别指横向和纵向的步长，第一个参数是ｂａｔｃｈ个数，第４个参数是卷积的深度，两者通常都设置为１
    conv = tf.nn.conv2d(images,kernel,[1,4,4,1],padding='SAME')
    #bias 的ｓｈａｐｅ等于该层的输出层维数
    biases = tf.Variable(tf.constant(0.0,shape=[64],dtype=tf.float32),trainable=True,name='biases')
    bias = tf.nn.bias_add(conv,biases)
    conv1 = tf.nn.relu(bias,name=scope)
    printActivation(conv1)
    
    parameters+=[kernel,biases]
    #AlexNet 使用最大池化而不是平均池化，避免了模糊
    #kernel size = 3x3 就是把3x3的像素块降为1x1的像素块 padding='VALID'，取样的时候不能超过边框，而'SAME'的时候可以填充边界外的点
    pool1 = tf.nn.max_pool(conv1,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID',name='pool1')
    
    #printActivation(pool1)
    
    
  with tf.name_scope('conv2') as scope:
    
    #卷积核尺寸为5x5，输入通道数（也就是上一层的输出通道数，也是上一层的卷积核数量）＝６４，这一层的卷积核数目为１９２
    kernel = tf.Variable(tf.truncated_normal([5,5,64,192],dtype=tf.float32,stddev=1e-1),name='weights')

    conv = tf.nn.conv2d(pool1,kernel,[1,1,1,1],padding='SAME')
    
    biases = tf.Variable(tf.constant(0.0,shape=[192],dtype=tf.float32),trainable=True,name='biases')
    bias = tf.nn.bias_add(conv,biases)
    conv2 = tf.nn.relu(bias,name=scope)
   
    
    parameters+=[kernel,biases]
    #AlexNet 使用最大池化而不是平均池化，避免了模糊
    #kernel size = 3x3 就是把3x3的像素块降为1x1的像素块 padding='VALID'，取样的时候不能超过边框，而'SAME'的时候可以填充边界外的点
    pool2 = tf.nn.max_pool(conv2,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID',name='pool2')
    
    
  with tf.name_scope('conv3') as scope:
    
    kernel = tf.Variable(tf.truncated_normal([3,3,192,384],dtype=tf.float32,stddev=1e-1),name='weights')
    conv = tf.nn.conv2d(pool2,kernel,[1,1,1,1],padding='SAME')
    biases = tf.Variable(tf.constant(0.0,shape=[384],dtype=tf.float32),trainable=True,name='biases')
    bias = tf.nn.bias_add(conv,biases)
    conv3 = tf.nn.relu(bias,name=scope)
    
    parameters+=[kernel,biases]
    
  with tf.name_scope('conv4') as scope:
    
    kernel = tf.Variable(tf.truncated_normal([3,3,384,256],dtype=tf.float32,stddev=1e-1),name='weights')
    conv = tf.nn.conv2d(conv3,kernel,[1,1,1,1],padding='SAME')
    biases = tf.Variable(tf.constant(0.0,shape=[256],dtype=tf.float32),trainable=True,name='biases')
    bias = tf.nn.bias_add(conv,biases)
    conv4 = tf.nn.relu(bias,name=scope)
    parameters+=[kernel,biases]
    
  with tf.name_scope('conv5') as scope:
    
    kernel = tf.Variable(tf.truncated_normal([3,3,256,256],dtype=tf.float32,stddev=1e-1),name='weights')
    conv = tf.nn.conv2d(conv4,kernel,[1,1,1,1],padding='SAME')
    biases = tf.Variable(tf.constant(0.0,shape=[256],dtype=tf.float32),trainable=True,name='biases')
    bias = tf.nn.bias_add(conv,biases)
    conv5 = tf.nn.relu(bias,name=scope)
    parameters+=[kernel,biases]
    
    pool5 = tf.nn.max_pool(conv5,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID',name='pool5')
    
    fcInput = tf.reshape(pool5,[-1,256*6*6])
    fc1 = fcLayer(fcInput,256*6*6,4096,True,"fc6")
    dropout1 = tf.nn.dropout(fc1,0.5)
    
    fc2 = fcLayer(dropout1,4096,4096,True,"fc7")
    dropout2 = tf.nn.dropout(fc2,0.5)
    
    #here 1000 means self.classnumber
    fc3 = fcLayer(dropout2,4096,1000,True,"fc8")
    
    
  return fc3,parameters
    

In [0]:
def time_tensorflow_run(session, target, info_string):
#  """Run the computation to obtain the target tensor and print timing stats.
#
#  Args:
#    session: the TensorFlow session to run the computation under.
#    target: the target Tensor that is passed to the session's run() function.
#    info_string: a string summarizing this run, to be printed with the stats.
#
#  Returns:
#    None
#  """
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target)
        duration = time.time() - start_time
        if i >= num_steps_burn_in:
            if not i % 10:
                print ('%s: step %d, duration = %.3f' %
                       (datetime.now(), i - num_steps_burn_in, duration))
            total_duration += duration
            total_duration_squared += duration * duration
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches - mn * mn
    sd = math.sqrt(vr)
    print ('%s: %s across %d steps, %.3f +/- %.3f sec / batch' %
           (datetime.now(), info_string, num_batches, mn, sd))



In [0]:
#running!
def run():
  with tf.Graph().as_default():
    image_size=224
    images = tf.Variable(tf.random_normal([batch_size,image_size,image_size,3],dtype=tf.float32,
                                         stddev=1e-1))
    fc3,parameters = inference(images)
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    time_tensorflow_run(sess,fc3,"Forward")
    objective = tf.nn.l2_loss(fc3)
    grad = tf.gradients(objective,parameters)
    time_tensorflow_run(sess,grad,"Forward-backward")

In [0]:
run()